In [31]:
pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [32]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [33]:
# Auth
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="394cfe3d8fe648f78e577788013d43d0",
    client_secret="efca9c50411f42bda9e4efc4a01ef824"
))

In [34]:
# 1. Chercher le morceau
track_name = "Lose Yourself"
results = sp.search(q=track_name, type="track", limit=1)
track_id = results["tracks"]["items"][0]["id"]


In [35]:
print(f"Track ID: {track_id}")

# 2️⃣ Récupérer toutes les infos sur la track
track_info = sp.track(track_id)
print("Nom:", track_info["name"])
print("Artistes:", [artist["name"] for artist in track_info["artists"]])
print("Album:", track_info["album"]["name"])
print("Date de sortie:", track_info["album"]["release_date"])
print("Durée (ms):", track_info["duration_ms"])
print("Popularité:", track_info["popularity"])
print("Lien Spotify:", track_info["external_urls"]["spotify"])
print("Preview URL:", track_info["preview_url"])

Track ID: 7MJQ9Nfxzh8LPZ9e9u68Fq
Nom: Lose Yourself
Artistes: ['Eminem']
Album: SHADYXV
Date de sortie: 2014-11-24
Durée (ms): 320626
Popularité: 71
Lien Spotify: https://open.spotify.com/track/7MJQ9Nfxzh8LPZ9e9u68Fq
Preview URL: None


In [ ]:
audio_features = sp.audio_features([track_id])[0]
print("Tempo:", audio_features["tempo"])
print("Energy:", audio_features["energy"])
print("Danceability:", audio_features["danceability"])
print("Key:", audio_features["key"])

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7MJQ9Nfxzh8LPZ9e9u68Fq with Params: {} returned 403 due to None


SpotifyException: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=7MJQ9Nfxzh8LPZ9e9u68Fq:
 None, reason: None